Standard UUIDs are nice and all, but why not extend the character range to include all standard ascii letters and numbers, also, get rid of embedded MAC addresses and time stamps.  

In [1]:
from secrets import token_urlsafe as TUS

class xuid:
    def __init__(self, TUS_length=64):
        initial_string = TUS(TUS_length).replace("_", "").replace("-", "")
        self.used_chars = initial_string[:32]
        segments = (initial_string[0:8],
                    initial_string[8:12],
                    initial_string[12:16],
                    initial_string[16:20],
                    initial_string[20:32])
        self.xuid_join = "-".join(segments)
    def __str__(self):
        return(self.xuid_join)
    def __int__(self):
        return(self.short_interizer())
    def __index__(self):
        return(self.short_interizer())
    def long_interizer(self):
        charlist = list(self.used_chars)
        ordlist = [ord(x) for x in charlist]
        binlist = [bin(x) for x in ordlist]
        strbin = [str(x)[2:] for x in binlist]
        strbin8 = [f"{'0' * (8-len(x))}{x}" for x in strbin]
        bstr = "0b" + "".join(strbin8)
        return(int(bstr, 2))
    def short_interizer(self):
        strlist = [str(bin(ord(x)))[2:] for x in list(self.used_chars)]
        return(int("0b" +  "".join([f"{'0' * (8-len(x))}{x}" for x in strlist]), 2))
        
    
print(str(xuid()))
int(xuid())
hex(xuid())

naSJps4K-61Q6-n94H-ynvg-nNzMUQ30BvUT


'0x7941567531766d7477496c55573855434e51465165584372554c755a6165334e'

The object is very simple, just a random string from `token_urlsafe`, and then sliced up to a format we can use.
Now let's figure out which method of creating the __int__ is faster, with a lot of variables or our two list comprehensions.

In [2]:
import timeit

def sf():
    xuid().short_interizer()
def lf():
    xuid().long_interizer()
st = timeit.timeit(lambda: sf(), number=10000000)
lt = timeit.timeit(lambda: lf(), number=10000000)
print(f"Short Time:   {st}s\nLong Time:    {lt}s")

Short Time:   494.6031452s
Long Time:    555.9972513s


Okay, so we can see that the short interizer is the faster method. As this is just a random series of characters, there's no reason to reinstantiate an older xuid. 